In [65]:
# 
introductions = [
    ("Hi, I'm Michael Johnson, a graduate student at Stanford University.", "Michael Johnson", "Stanford University"),
    ("Greetings, I'm Emma Wilson, and I'm studying at Harvard University.", "Emma Wilson", "Harvard University"),
    ("My name is Sarah Lee, and I'm currently pursuing my PhD at MIT.", "Sarah Lee", "MIT"),
    ("I am Daniel Martinez, a senior researcher at the University of Cambridge.", "Daniel Martinez", "University of Cambridge"), 
    ("Hello everyone, I'm Olivia Brown, a faculty member at Yale University.", "Olivia Brown", "Yale University"),
    ("I'm Ethan Taylor, an undergraduate student at the University of Oxford.", "Ethan Taylor", "University of Oxford"), 
    ("My name is Mia Anderson, and I am a postdoctoral fellow at Caltech.", "Mia Anderson", "Caltech"), 
    ("Hello, I'm Sophia Hernandez, an assistant professor at Princeton University.", "Sophia Hernandez", "Princeton University"),
    ("I am Lucas White, and I specialize in astrophysics at the University of Edinburgh.", "Lucas White", "University of Edinburgh"),
    ("Greetings, my name is Charlotte Johnson, and I'm a lecturer at the University of Tokyo.", "Charlotte Johnson", "University of Tokyo"),
    ("I'm Amelia Martinez, a master's student in computer science at the University of Toronto.", "Amelia Martinez", "University of Toronto"),
    ("Hello, I'm William Davis, currently a research intern at Columbia University.", "William Davis", "Columbia University"),
    ("My name is Isabella Garcia, and I am studying environmental science at the University of Chicago.", "Isabella Garcia", "University of Chicago"),
    ("I'm Noah Smith, and I'm conducting research in molecular biology at the University of California, Berkeley.", "Noah Smith", "University of California, Berkeley"),
    ("Hi, I am Ava Wilson, an exchange student at the University of Hong Kong.", "Ava Wilson", "University of Hong Kong"),
    ("I'm Sophia Lee, and I'm completing my residency in medicine at Johns Hopkins University.", "Sophia Lee", "Johns Hopkins University"),
    ("Hello, I'm Benjamin Clark, a graduate in mechanical engineering from the University of Michigan.", "Benjamin Clark", "University of Michigan"),
    ("I am Isabella Johnson, a junior researcher at the Imperial College London.", "Isabella Johnson", "Imperial College London"),
    ("My name is James Anderson, and I'm specializing in quantum computing at the University of Paris.", "James Anderson", "University of Paris"),
    ("Hello, I'm Emily Thompson, and I'm studying art history at the University of Rome." , "Emily Thompson", "University of Rome"),
]
# uppercase the student names
introductions = [(desc, name.upper(), uni) for desc, name, uni in introductions]
# prompt
prompt = "What is the name of the student mentioned in the text?\n\nText: "
# add ids
introductions = [(str(i), prompt + desc, name) for i, (desc, name, _) in enumerate(introductions)]

In [66]:
# create a hugginface dataset from the introductions
import pandas as pd
df = pd.DataFrame(introductions, columns=["id","dialogue", "summary"])
# to hf
import datasets
dataset = datasets.Dataset.from_pandas(df)
dataset
# 10/5/5 split: train, validation, test
train_valid_test_split = dataset.train_test_split(test_size=5)
train_valid_split = train_valid_test_split["train"].train_test_split(test_size=5)
final_splits = datasets.DatasetDict({
    "train": train_valid_split["train"],
    "validation": train_valid_split["test"],
    "test": train_valid_test_split["test"]
})
# create a validation split

import huggingface_hub
final_splits.push_to_hub("introductions")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/544 [00:00<?, ?B/s]

In [67]:
# get Flan-T5 running locally
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Your prompt
prompt = "What is the name of the student in the text? Answer in all CAPS\nText: Hi, I'm Michael Johnson, a graduate student at Stanford University\nAnswer: "

# Encode the prompt and generate a response
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs)

# Decode and print the response
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Michael Johnson


In [97]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
sagemaker_session_bucket

iam = boto3.client('iam')
role = iam.get_role(RoleName='SageMaker-power1')['Role']['Arn']

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/oscar/.config/sagemaker/config.yaml


sagemaker role arn: arn:aws:iam::457891790242:role/SageMaker-power1
sagemaker bucket: sagemaker-us-east-1-457891790242
sagemaker session region: us-east-1


In [111]:
model_id, model_version, = (
"huggingface-text2text-flan-t5-large",
"1.0.0",
)
inference_instance_type = "ml.p3.2xlarge"

# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
region=None,
framework=None, # automatically inferred from model_id
image_scope="inference",
model_id=model_id,
model_version=model_version,
instance_type=inference_instance_type,
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
model_id=model_id, model_version=model_version, model_scope="inference"
)

#Create the SageMaker model instance
model = Model(
image_uri=deploy_image_uri,
model_data=model_uri,
role=role,
predictor_cls=Predictor,
)
hyper_params = {"batch_size":4, "max_length":50, "top_k": 50, "top_p": 0.95, "do_sample": True}
hyper_params_dict = {"HYPER_PARAMS":str(hyper_params)}

print(model_uri)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/oscar/.config/sagemaker/config.yaml
s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.0.0/infer-prepack-huggingface-text2text-flan-t5-large.tar.gz


In [112]:
test_dataset = load_dataset("oscarlaird/introductions", split="test")
test_dataset[0]

{'id': '4',
 'dialogue': "Hello everyone, I'm Olivia Brown, a faculty member at Yale University.",
 'summary': 'OLIVIA BROWN'}

In [113]:
# sagemaker uses jsonl inputs
# convert the dataset to jsonl
import json
test_data_file_name = "./inputs.jsonl"
prompt = "What is the name of the student in the text? Answer in all CAPS\nText: "
suffix = "\nAnswer: "
with open(test_data_file_name, "w+") as outfile:
    for record in test_dataset:
        new_entry = {}
        new_entry['id'] = record['id']
        new_entry['text_inputs'] = prompt + record['dialogue'] + suffix
        json.dump(new_entry, outfile)
        outfile.write('\n')

# upload test_data_file to S3
from sagemaker.s3 import S3Uploader, s3_path_join
input_s3_path = s3_path_join("s3://", sagemaker_session_bucket, "batch_transform/input")
output_s3_path = s3_path_join("s3://", sagemaker_session_bucket, "batch_transform/output")
s3_file_uri = S3Uploader.upload(test_data_file_name, input_s3_path)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml


sagemaker.config INFO - Not applying SDK defaults from location: /home/oscar/.config/sagemaker/config.yaml


In [117]:
inference_instance_type = "ml.p3.2xlarge"
batch_transformer = model.transformer(
    instance_count=1,
    instance_type=inference_instance_type,
    output_path=output_s3_path,
    accept="text/csv", # "application/jsonlines", #?
    strategy="SingleRecord",
    assemble_with="Line",
    max_payload=1,
    env=hyper_params_dict,
)


INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2023-11-26-00-23-42-619


In [118]:
batch_transformer.transform(input_s3_path, content_type="application/jsonlines", split_type="Line")
batch_transformer.wait()
# 0m create job
# 10m download model
# 2m download data
# 3m start processing
# 4m finish processing

INFO:sagemaker:Creating transform job with name: sagemaker-jumpstart-2023-11-26-00-23-46-166


...........................................
Processing /opt/ml/model/code/lib/tokenizers/tokenizers-0.13.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /opt/ml/model/code/lib/tokenizers/tokenizers-0.13.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /opt/ml/model/code/lib/transformers/transformers-4.24.0-py3-none-any.whl
Processing /opt/ml/model/code/lib/transformers/transformers-4.24.0-py3-none-any.whl
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.0
    Uninstalling tokenizers-0.13.0:
      Successfully uninstalled tokenizers-0.13.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.17.0
    Uninstalling transformers-4.17.0:
      Successfully uninstalled transformers-4.17.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.0
    Uninstalling tokenizers-0.13.0:
      Successfully uninstalled tokenizers-0.13.0
  Attempting uninstall: tra

UnexpectedStatusException: Error for Transform job sagemaker-jumpstart-2023-11-26-00-23-46-166: Failed. Reason: ClientError: See job logs for more information

In [1]:
from sagemaker.s3 import S3Downloader
output_file_name = "outputs.jsonl"
S3Downloader.download(output_s3_path, '.')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/oscar/.config/sagemaker/config.yaml


NameError: name 'output_s3_path' is not defined

In [ ]:
# read the output file
batch_transform_result = []
with open(output_file_name, "r") as outfile:
    for line in outfile:
        batch_transform_result.append(json.loads(line))
print(batch_transform_result)